## Setup

**For ODU HPC (Wahab):**
```bash
module load container_env cuda-12.3.0
crun -p ~/envs/cuda python -m jupyter lab
```

In [ ]:
import numpy as np
from numba import cuda
import math
import time

# Verify GPU
print(f"CUDA available: {cuda.is_available()}")
if cuda.is_available():
    device = cuda.get_current_device()
    print(f"Device: {device.name}")
    print(f"Compute capability: {device.compute_capability}")

---

## Part 1: Complete CUDA Memory Hierarchy Review

Before diving into special memory types, let's review the complete memory hierarchy:

```
┌─────────────────────────────────────────────────────────────┐
│                    CUDA Memory Hierarchy                    │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│  ┌─────────────────────────────────────────────────────┐   │
│  │              Global Memory (GB, slowest)            │   │
│  │  • All threads can read/write                       │   │
│  │  • Persists for application lifetime                │   │
│  │  • ~400-900 GB/s bandwidth                          │   │
│  └─────────────────────────────────────────────────────┘   │
│                          │                                  │
│  ┌──────────────────────┬┴──────────────────────┐          │
│  │                      │                        │          │
│  ▼                      ▼                        ▼          │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐      │
│  │   Constant   │  │   Texture    │  │    Shared    │      │
│  │    Memory    │  │    Memory    │  │    Memory    │      │
│  │  (64KB,      │  │  (cached,    │  │  (48-164KB   │      │
│  │   cached)    │  │   read-only) │  │   per SM)    │      │
│  └──────────────┘  └──────────────┘  └──────────────┘      │
│         │                 │                  │              │
│         └─────────────────┼──────────────────┘              │
│                           │                                  │
│                           ▼                                  │
│            ┌──────────────────────────────┐                 │
│            │  Registers (fastest, private) │                 │
│            │  • Per-thread, ~256 per thread│                 │
│            │  • ~12 TB/s equivalent        │                 │
│            └──────────────────────────────┘                 │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Memory Types Summary

| Memory Type | Location | Scope | Lifetime | Cache | Speed |
|-------------|----------|-------|----------|-------|-------|
| Register | On-chip | Thread | Thread | N/A | Fastest |
| Local | Off-chip | Thread | Thread | L1/L2 | Slow |
| Shared | On-chip | Block | Block | N/A | Fast |
| Global | Off-chip | Grid | Application | L1/L2 | Slow |
| Constant | Off-chip | Grid | Application | Constant cache | Fast (broadcast) |
| Texture | Off-chip | Grid | Application | Texture cache | Fast (spatial) |

---

## Part 2: Constant Memory

### What is Constant Memory?

Constant memory is a **read-only** memory space that:
- Has 64KB total capacity per GPU
- Is cached in a dedicated **constant cache**
- Optimized for **broadcasting** same value to all threads
- Initialized by the host before kernel launch

### When Constant Memory Excels

```
BEST CASE: All threads read the SAME address
┌───────────────────────────────────────────────────┐
│  Thread 0  Thread 1  Thread 2  ...  Thread 31    │
│     │         │         │              │         │
│     └─────────┴─────────┴──────────────┘         │
│                     │                             │
│                     ▼                             │
│           ┌─────────────────┐                    │
│           │ Constant Cache  │                    │
│           │   (1 read)      │                    │
│           └─────────────────┘                    │
│  → Result broadcast to all 32 threads in 1 cycle │
└───────────────────────────────────────────────────┘

WORST CASE: All threads read DIFFERENT addresses
┌───────────────────────────────────────────────────┐
│  Thread 0  Thread 1  Thread 2  ...  Thread 31    │
│     │         │         │              │         │
│     ▼         ▼         ▼              ▼         │
│   addr[0]  addr[1]  addr[2]  ...  addr[31]      │
│     │         │         │              │         │
│     └─────────┴─────────┴──────────────┘         │
│                     │                             │
│                     ▼                             │
│           ┌─────────────────┐                    │
│           │ 32 serial reads │ ← SLOW!            │
│           └─────────────────┘                    │
└───────────────────────────────────────────────────┘
```

### Ideal Use Cases for Constant Memory

1. **Filter kernels/masks** (convolution, Gaussian blur)
2. **Lookup tables** accessed uniformly
3. **Configuration parameters** (dimensions, coefficients)
4. **Mathematical constants** (π, e, conversion factors)

### Example: Convolution with Constant Memory

Image convolution is the **classic** use case for constant memory because all threads apply the **same filter kernel**.

In [ ]:
# 3x3 Sobel edge detection filter
SOBEL_X = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)

print("Sobel X filter (edge detection):")
print(SOBEL_X)

In [ ]:
# Version 1: Passing filter as regular global memory
@cuda.jit
def convolution_global(image, filter_kernel, output):
    """Convolution using global memory for filter."""
    row, col = cuda.grid(2)
    height, width = image.shape
    
    if row > 0 and row < height - 1 and col > 0 and col < width - 1:
        result = 0.0
        for i in range(-1, 2):
            for j in range(-1, 2):
                # Each read of filter_kernel goes to global memory
                result += image[row + i, col + j] * filter_kernel[i + 1, j + 1]
        output[row, col] = result

In [ ]:
# In Numba, we simulate constant memory behavior by:
# 1. Using a closure to capture the filter at compile time
# 2. Or placing filter in shared memory (next best thing)

# Version 2: Filter in shared memory (simulating constant memory)
@cuda.jit
def convolution_shared_filter(image, filter_kernel, output):
    """Convolution with filter loaded to shared memory."""
    # Load filter to shared memory (done once per block)
    shared_filter = cuda.shared.array((3, 3), dtype=np.float32)
    
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y
    
    # First 9 threads load the filter
    linear_tid = ty * cuda.blockDim.x + tx
    if linear_tid < 9:
        fi, fj = linear_tid // 3, linear_tid % 3
        shared_filter[fi, fj] = filter_kernel[fi, fj]
    
    cuda.syncthreads()
    
    row, col = cuda.grid(2)
    height, width = image.shape
    
    if row > 0 and row < height - 1 and col > 0 and col < width - 1:
        result = 0.0
        for i in range(-1, 2):
            for j in range(-1, 2):
                # Filter reads now from fast shared memory
                result += image[row + i, col + j] * shared_filter[i + 1, j + 1]
        output[row, col] = result

In [ ]:
# Test both versions
SIZE = 2048
image = np.random.rand(SIZE, SIZE).astype(np.float32)
output = np.zeros_like(image)

d_image = cuda.to_device(image)
d_filter = cuda.to_device(SOBEL_X)
d_output = cuda.to_device(output)

threads = (16, 16)
blocks = ((SIZE + 15) // 16, (SIZE + 15) // 16)

# Warm up
convolution_global[blocks, threads](d_image, d_filter, d_output)
convolution_shared_filter[blocks, threads](d_image, d_filter, d_output)
cuda.synchronize()

# Benchmark global memory filter
start = time.perf_counter()
for _ in range(100):
    convolution_global[blocks, threads](d_image, d_filter, d_output)
cuda.synchronize()
global_time = (time.perf_counter() - start) / 100 * 1000

# Benchmark shared memory filter
start = time.perf_counter()
for _ in range(100):
    convolution_shared_filter[blocks, threads](d_image, d_filter, d_output)
cuda.synchronize()
shared_time = (time.perf_counter() - start) / 100 * 1000

print(f"Image size: {SIZE}x{SIZE}")
print(f"Global memory filter: {global_time:.3f} ms")
print(f"Shared memory filter: {shared_time:.3f} ms")
print(f"Speedup: {global_time/shared_time:.2f}x")

### Understanding Constant Memory Benefits

Even though Numba doesn't have direct constant memory syntax, the concept matters:

```
Why constant memory is fast for filters:

For a 3x3 convolution:
- Each thread reads the SAME 9 filter values
- 1 warp = 32 threads all reading filter[0,0]
- With constant cache: 1 read, broadcast to 32
- Without: 32 reads to global memory

Result: 32x reduction in filter memory traffic!
```

In native CUDA C++:
```cpp
__constant__ float filter[9];  // Declared at file scope

// Copied before kernel launch
cudaMemcpyToSymbol(filter, host_filter, 9 * sizeof(float));
```

---

## Part 3: Texture Memory

### What is Texture Memory?

Texture memory is another **read-only** memory that:
- Is optimized for **2D spatial locality**
- Has a dedicated **texture cache**
- Supports **hardware interpolation** (free bilinear/trilinear)
- Supports **automatic boundary handling** (clamp, wrap, mirror)
- Originally designed for graphics, but useful for compute

### Texture Cache vs L1 Cache

```
L1 Cache: Optimized for 1D linear access (coalescing)
┌────────────────────────────────────────┐
│ [0][1][2][3][4][5][6][7] → cache line  │
│ Linear memory layout                    │
└────────────────────────────────────────┘

Texture Cache: Optimized for 2D spatial access
┌────────────────────────────────────────┐
│ [0,0][0,1]│[0,2][0,3]                  │
│ [1,0][1,1]│[1,2][1,3]  → 2D tiles      │
│ ─────────┼───────────                  │
│ [2,0][2,1]│[2,2][2,3]                  │
│ [3,0][3,1]│[3,2][3,3]                  │
└────────────────────────────────────────┘
```

### When Texture Memory Excels

1. **Image processing** (resizing, rotation, warping)
2. **Interpolation** operations
3. **Random 2D access patterns**
4. **Volume rendering** (3D textures)
5. **Data with spatial locality** in 2D/3D

### Texture Memory in Modern CUDA

Modern GPUs use **texture objects** (introduced in CUDA 5.0):

```cpp
// Create texture object in CUDA C++
cudaResourceDesc resDesc;
resDesc.resType = cudaResourceTypeArray;
resDesc.res.array.array = cudaArray;

cudaTextureDesc texDesc;
texDesc.addressMode[0] = cudaAddressModeClamp;  // Boundary handling
texDesc.filterMode = cudaFilterModeLinear;       // Interpolation
texDesc.normalizedCoords = true;                 // Use [0,1] coords

cudaTextureObject_t tex;
cudaCreateTextureObject(&tex, &resDesc, &texDesc, NULL);

// In kernel: automatic interpolation!
float value = tex2D<float>(tex, x + 0.5f, y + 0.5f);  // Gets interpolated value
```

Unfortunately, Numba CUDA doesn't directly support texture objects. For texture-like benefits in Numba:
1. Use shared memory tiling for 2D spatial locality
2. Implement manual interpolation
3. For advanced cases, use CuPy or raw CUDA

### Simulating Texture Benefits: Image Interpolation

In [ ]:
# Manual bilinear interpolation (what texture memory does for free)
@cuda.jit(device=True)
def bilinear_sample(image, x, y, height, width):
    """Bilinear interpolation at floating-point coordinates."""
    # Clamp to valid range
    x = max(0.0, min(x, width - 1.001))
    y = max(0.0, min(y, height - 1.001))
    
    # Get integer coordinates
    x0 = int(x)
    y0 = int(y)
    x1 = min(x0 + 1, width - 1)
    y1 = min(y0 + 1, height - 1)
    
    # Fractional parts
    fx = x - x0
    fy = y - y0
    
    # Bilinear interpolation
    v00 = image[y0, x0]
    v01 = image[y0, x1]
    v10 = image[y1, x0]
    v11 = image[y1, x1]
    
    v0 = v00 * (1 - fx) + v01 * fx
    v1 = v10 * (1 - fx) + v11 * fx
    
    return v0 * (1 - fy) + v1 * fy

In [ ]:
# Image resize using bilinear interpolation
@cuda.jit
def resize_bilinear(src, dst):
    """Resize image using bilinear interpolation."""
    dst_y, dst_x = cuda.grid(2)
    dst_h, dst_w = dst.shape
    src_h, src_w = src.shape
    
    if dst_y < dst_h and dst_x < dst_w:
        # Map destination coords to source coords
        src_x = dst_x * (src_w - 1) / (dst_w - 1)
        src_y = dst_y * (src_h - 1) / (dst_h - 1)
        
        dst[dst_y, dst_x] = bilinear_sample(src, src_x, src_y, src_h, src_w)

In [ ]:
# Test image resizing
src_size = 256
dst_size = 512

# Create a simple test pattern
src_image = np.zeros((src_size, src_size), dtype=np.float32)
for i in range(src_size):
    for j in range(src_size):
        src_image[i, j] = (i + j) / (2 * src_size)

dst_image = np.zeros((dst_size, dst_size), dtype=np.float32)

d_src = cuda.to_device(src_image)
d_dst = cuda.to_device(dst_image)

threads = (16, 16)
blocks = ((dst_size + 15) // 16, (dst_size + 15) // 16)

resize_bilinear[blocks, threads](d_src, d_dst)
result = d_dst.copy_to_host()

print(f"Resized from {src_size}x{src_size} to {dst_size}x{dst_size}")
print(f"Source range: [{src_image.min():.3f}, {src_image.max():.3f}]")
print(f"Result range: [{result.min():.3f}, {result.max():.3f}]")
print("\nNote: With texture memory, interpolation would be automatic!")

---

## Part 4: Memory Type Decision Guide

### Decision Flowchart

```
                 ┌──────────────────────┐
                 │ Need to store data   │
                 │   for GPU kernel?    │
                 └──────────┬───────────┘
                            │
              ┌─────────────┴─────────────┐
              ▼                           ▼
   ┌────────────────────┐    ┌────────────────────┐
   │ Read-only data?    │    │ Read-write data?   │
   └────────┬───────────┘    └────────┬───────────┘
            │                          │
   ┌────────┴────────┐         ┌───────┴────────┐
   ▼                 ▼         ▼                ▼
┌───────┐       ┌────────┐  ┌────────┐   ┌───────────┐
│ Small │       │ Large  │  │Private │   │  Shared   │
│ <64KB │       │data or │  │to each │   │among block│
│uniform│       │spatial │  │thread  │   │threads    │
│access │       │access  │  │        │   │           │
└───┬───┘       └───┬────┘  └───┬────┘   └─────┬─────┘
    │               │           │              │
    ▼               ▼           ▼              ▼
┌────────┐   ┌──────────┐  ┌────────┐   ┌───────────┐
│CONSTANT│   │ TEXTURE  │  │REGISTER│   │  SHARED   │
│ MEMORY │   │  MEMORY  │  │(auto)  │   │  MEMORY   │
└────────┘   └──────────┘  └────────┘   └───────────┘

Default: GLOBAL MEMORY (with coalescing optimizations)
```

### Quick Reference Table

| Scenario | Best Memory | Why |
|----------|-------------|-----|
| Convolution kernel/filter | Constant | Same values read by all threads |
| Configuration parameters | Constant | Small, uniform read access |
| Lookup table (uniform access) | Constant | Broadcast efficiency |
| Image processing (resize, rotate) | Texture | 2D spatial locality + interpolation |
| Volume rendering | Texture | 3D spatial locality |
| Random 2D reads | Texture | 2D cache optimization |
| Thread-local accumulator | Register | Fastest, private to thread |
| Block-wide reduction | Shared | Threads need to communicate |
| Tiled matrix multiply | Shared | Data reuse within block |
| Histogram (atomic updates) | Shared → Global | Reduce atomic contention |
| Large arrays with streaming | Global | Only option for large data |

---

## Part 5: Practical Example - Optimized Gaussian Blur

Let's combine all Week 2 concepts into a complete optimized implementation.

In [ ]:
def gaussian_kernel_2d(size, sigma):
    """Generate 2D Gaussian kernel."""
    ax = np.arange(-size // 2 + 1, size // 2 + 1)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2 * sigma**2))
    return (kernel / kernel.sum()).astype(np.float32)

# 5x5 Gaussian kernel
KERNEL_SIZE = 5
GAUSSIAN = gaussian_kernel_2d(KERNEL_SIZE, 1.0)
print("5x5 Gaussian kernel:")
print(np.round(GAUSSIAN, 4))

In [ ]:
# Naive implementation: Global memory only
@cuda.jit
def gaussian_blur_naive(image, kernel, output, ksize):
    """Naive Gaussian blur - all global memory."""
    row, col = cuda.grid(2)
    height, width = image.shape
    half_k = ksize // 2
    
    if row >= half_k and row < height - half_k and col >= half_k and col < width - half_k:
        result = 0.0
        for i in range(-half_k, half_k + 1):
            for j in range(-half_k, half_k + 1):
                result += image[row + i, col + j] * kernel[i + half_k, j + half_k]
        output[row, col] = result

In [ ]:
# Optimized implementation: Shared memory tiling + kernel in shared memory
TILE_SIZE = 16
BLOCK_SIZE = TILE_SIZE + KERNEL_SIZE - 1  # Tile + halo

@cuda.jit
def gaussian_blur_optimized(image, kernel, output, ksize):
    """Optimized Gaussian blur with shared memory tiling."""
    # Shared memory for image tile (with halo) and kernel
    shared_tile = cuda.shared.array((BLOCK_SIZE, BLOCK_SIZE), dtype=np.float32)
    shared_kernel = cuda.shared.array((5, 5), dtype=np.float32)
    
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y
    bx, by = cuda.blockIdx.x, cuda.blockIdx.y
    height, width = image.shape
    half_k = ksize // 2
    
    # Load kernel to shared memory (first 25 threads)
    linear_tid = ty * cuda.blockDim.x + tx
    if linear_tid < ksize * ksize:
        ki, kj = linear_tid // ksize, linear_tid % ksize
        shared_kernel[ki, kj] = kernel[ki, kj]
    
    # Calculate tile starting position (with halo offset)
    tile_start_row = by * TILE_SIZE - half_k
    tile_start_col = bx * TILE_SIZE - half_k
    
    # Load tile with halo into shared memory
    # Each thread may need to load multiple elements
    for i in range(0, BLOCK_SIZE, TILE_SIZE):
        for j in range(0, BLOCK_SIZE, TILE_SIZE):
            si = ty + i
            sj = tx + j
            if si < BLOCK_SIZE and sj < BLOCK_SIZE:
                gi = tile_start_row + si
                gj = tile_start_col + sj
                if 0 <= gi < height and 0 <= gj < width:
                    shared_tile[si, sj] = image[gi, gj]
                else:
                    shared_tile[si, sj] = 0.0
    
    cuda.syncthreads()
    
    # Compute output
    out_row = by * TILE_SIZE + ty
    out_col = bx * TILE_SIZE + tx
    
    if out_row < height and out_col < width:
        result = 0.0
        for i in range(ksize):
            for j in range(ksize):
                result += shared_tile[ty + i, tx + j] * shared_kernel[i, j]
        output[out_row, out_col] = result

In [ ]:
# Benchmark comparison
SIZE = 2048
image = np.random.rand(SIZE, SIZE).astype(np.float32)
output = np.zeros_like(image)

d_image = cuda.to_device(image)
d_kernel = cuda.to_device(GAUSSIAN)
d_output = cuda.to_device(output)

# Naive version
threads_naive = (16, 16)
blocks_naive = ((SIZE + 15) // 16, (SIZE + 15) // 16)

gaussian_blur_naive[blocks_naive, threads_naive](d_image, d_kernel, d_output, KERNEL_SIZE)
cuda.synchronize()

start = time.perf_counter()
for _ in range(50):
    gaussian_blur_naive[blocks_naive, threads_naive](d_image, d_kernel, d_output, KERNEL_SIZE)
cuda.synchronize()
naive_time = (time.perf_counter() - start) / 50 * 1000

# Optimized version
threads_opt = (TILE_SIZE, TILE_SIZE)
blocks_opt = ((SIZE + TILE_SIZE - 1) // TILE_SIZE, (SIZE + TILE_SIZE - 1) // TILE_SIZE)

gaussian_blur_optimized[blocks_opt, threads_opt](d_image, d_kernel, d_output, KERNEL_SIZE)
cuda.synchronize()

start = time.perf_counter()
for _ in range(50):
    gaussian_blur_optimized[blocks_opt, threads_opt](d_image, d_kernel, d_output, KERNEL_SIZE)
cuda.synchronize()
optimized_time = (time.perf_counter() - start) / 50 * 1000

print(f"\n{'='*50}")
print(f"Gaussian Blur Performance ({SIZE}x{SIZE} image)")
print(f"{'='*50}")
print(f"Naive (global memory):     {naive_time:.3f} ms")
print(f"Optimized (shared memory): {optimized_time:.3f} ms")
print(f"Speedup:                   {naive_time/optimized_time:.2f}x")
print(f"\nOptimizations applied:")
print("  ✓ Kernel loaded to shared memory (constant-like behavior)")
print("  ✓ Image tile with halo in shared memory")
print("  ✓ Coalesced global memory loads")

---

## Exercises

### Exercise 1: Separable Gaussian Blur

A 2D Gaussian is **separable**: it can be computed as two 1D passes (horizontal then vertical). This reduces operations from O(k²) to O(2k).

In [ ]:
# 1D Gaussian kernel
def gaussian_kernel_1d(size, sigma):
    ax = np.arange(-size // 2 + 1, size // 2 + 1)
    kernel = np.exp(-ax**2 / (2 * sigma**2))
    return (kernel / kernel.sum()).astype(np.float32)

GAUSSIAN_1D = gaussian_kernel_1d(5, 1.0)
print("1D Gaussian kernel:", GAUSSIAN_1D)

In [ ]:
# TODO: Implement horizontal and vertical 1D convolution kernels

@cuda.jit
def gaussian_blur_horizontal(image, kernel, output, ksize):
    """Apply 1D Gaussian blur horizontally."""
    # Your implementation here
    pass

@cuda.jit
def gaussian_blur_vertical(image, kernel, output, ksize):
    """Apply 1D Gaussian blur vertically."""
    # Your implementation here
    pass

# Compare:
# 1. 5x5 2D convolution: 25 multiplications per pixel
# 2. Two 1x5 1D convolutions: 10 multiplications per pixel
# Expected speedup: ~2.5x from reduced arithmetic

### Exercise 2: Lookup Table with Constant Memory Pattern

Implement a color mapping kernel where all threads read from the same lookup table.

In [ ]:
# Create a colormap lookup table (256 entries)
# Maps grayscale values to "heat" colors
def create_heatmap_lut():
    lut = np.zeros((256, 3), dtype=np.float32)
    for i in range(256):
        t = i / 255.0
        # Blue -> Cyan -> Green -> Yellow -> Red
        if t < 0.25:
            lut[i] = [0, t * 4, 1]
        elif t < 0.5:
            lut[i] = [0, 1, 1 - (t - 0.25) * 4]
        elif t < 0.75:
            lut[i] = [(t - 0.5) * 4, 1, 0]
        else:
            lut[i] = [1, 1 - (t - 0.75) * 4, 0]
    return lut

HEATMAP_LUT = create_heatmap_lut()
print(f"Lookup table shape: {HEATMAP_LUT.shape}")

In [ ]:
# TODO: Implement color mapping with LUT in shared memory

@cuda.jit
def apply_heatmap(grayscale, lut, output_rgb):
    """
    Apply heatmap colorization using lookup table.
    
    Args:
        grayscale: 2D input (H, W), values 0-255
        lut: Lookup table (256, 3)
        output_rgb: 3D output (H, W, 3)
    
    Hint: Load LUT to shared memory for constant-memory-like behavior
    """
    # Your implementation here
    pass

---

## Week 2 Summary

### Key Concepts Mastered

| Day | Topic | Key Takeaway |
|-----|-------|-------------|
| 1 | Memory Coalescing | Adjacent threads → adjacent memory = fast |
| 2 | Shared Memory | On-chip memory for data reuse |
| 3 | Bank Conflicts | 32 banks, avoid same-bank access |
| 4 | Special Memory | Constant (broadcast), Texture (2D spatial) |

### Optimization Hierarchy

```
1. First: Choose right algorithm (parallelizable)
2. Then:  Ensure coalesced global memory access
3. Then:  Use shared memory for data reuse
4. Then:  Avoid bank conflicts (padding)
5. Then:  Consider special memory types
6. Then:  Fine-tune thread/block configuration
```

### Memory Selection Quick Guide

```
Small read-only + uniform access → Constant
2D spatial access + interpolation → Texture  
Block-local data reuse           → Shared
Thread-private temporary         → Register
Everything else                  → Global (with coalescing)
```

---

## Next Steps

✅ Completed Week 2: Memory Patterns & Optimization

📋 Day 5: Review & complete checkpoint quiz

📋 Week 3 Preview: **Synchronization & Atomics**
- Thread synchronization primitives
- Atomic operations
- Warp-level programming
- Parallel reduction patterns